In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from scipy.signal import find_peaks

In [ ]:
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\On-Chip Backward Stimulated Brillouin Scattering in Lithium Niobate Waveguides\\Data"

directory_file_list = os.listdir(directory)
selected_files = []
counter = 0

for i in directory_file_list:
    if i.endswith(".csv"):
        print(f"{counter}: {i}")
        selected_files.append(i)
        counter += 1

In [ ]:
idx_file = 0

file_name = selected_files[idx_file]
df_raw = pd.read_csv(directory+"\\"+selected_files[idx_file], engine='pyarrow')
columns_raw=df_raw.columns

print(file_name)
df_raw.head()

In [ ]:
idx_HCN_column = 2
idx_MZI_column = 1
#----------

HCN_raw = df_raw[columns_raw[idx_HCN_column]].values
MZI_raw = df_raw[columns_raw[idx_MZI_column]].values

fig, ax = plt.subplots(2,1,figsize=(12,4))

ax[0].plot(HCN_raw)
ax[0].set_ylabel(columns_raw[idx_HCN_column])
ax[0].set_xticks(np.linspace(0, len(HCN_raw),14))
ax[0].grid(True)

ax[1].plot(MZI_raw)
ax[1].set_ylabel(columns_raw[idx_MZI_column])
ax[1].set_xticks(np.linspace(0, len(MZI_raw),14))
ax[1].grid(True)

print(file_name)

In [ ]:
idx_crop_i = int(2.581e6)
idx_crop_f = int(6.4e6)
#----------

idx_crop_vec = np.arange(idx_crop_i, idx_crop_f, 1)

fig, ax = plt.subplots(2,1, figsize=(12,4))

ax[0].plot(HCN_raw, alpha=0.5)
ax[0].plot(idx_crop_vec, HCN_raw[idx_crop_i:idx_crop_f], c='tab:blue')
ax[0].plot([idx_crop_i, idx_crop_i], [np.min(HCN_raw), np.max(HCN_raw)], '--', c='r')
ax[0].plot([idx_crop_f, idx_crop_f], [np.min(HCN_raw), np.max(HCN_raw)], '--', c='r')
ax[0].set_xticks(np.linspace(0, len(HCN_raw),14))
ax[0].grid(True)
ax[0].set_ylabel(columns_raw[idx_HCN_column])

ax[1].plot(MZI_raw, alpha=0.5)
ax[1].plot(idx_crop_vec, MZI_raw[idx_crop_i:idx_crop_f], c='tab:blue')
ax[1].plot([idx_crop_i, idx_crop_i], [np.min(MZI_raw), np.max(MZI_raw)], '--', c='r')
ax[1].plot([idx_crop_f, idx_crop_f], [np.min(MZI_raw), np.max(MZI_raw)], '--', c='r')
ax[1].set_xticks(np.linspace(0, len(MZI_raw),14))
ax[1].grid(True)
ax[1].set_ylabel(columns_raw[idx_MZI_column])
ax[1].set_xlabel("Index Number")

print(file_name)

In [ ]:
HCN_crop = HCN_raw[idx_crop_i:idx_crop_f]
MZI_crop = MZI_raw[idx_crop_i:idx_crop_f]

HCN_min_threshold = 6
HCN_mean_idx_distance = 10000
HCN_values = [1543.11423, 1543.80967, 1544.51503, 1545.23033, 1545.95549,
              1546.69055, 1547.43558, 1548.19057, 1548.95555, 1549.73051,
              1550.51546, 1551.31045, 1552.11546, 1552.93051, 1553.75562,
              1554.59079, 1555.43605, 1556.29141, 1557.15686, 1558.03240,
              1558.91808, 1559.81389]
#-------------

idx_HCN_crop_mins = find_peaks(-HCN_crop, height=-HCN_min_threshold, distance=HCN_mean_idx_distance)[0]

fig, ax = plt.subplots(figsize=(14,4))

ax.plot(HCN_crop)
ax.scatter(idx_HCN_crop_mins, HCN_crop[idx_HCN_crop_mins], c='r', zorder=20)
ax.grid(True)
ax.set_ylabel("HCN")
ax.set_xlabel("Index Number")

for i in range(len(HCN_values)):
   plt.annotate(str(i)+" - "+str(idx_HCN_crop_mins[i])+" - "+str(HCN_values[i]), xy=(idx_HCN_crop_mins[i], HCN_crop[idx_HCN_crop_mins][i]), size=7, rotation=-90)

print(file_name)

In [ ]:
MZI_min_threshold = 0.02
MZI_max_threshold = 0.035
MZI_mean_idx_distance = 100

idx_MZI_maxs = find_peaks(MZI_crop, height=MZI_max_threshold, distance=MZI_mean_idx_distance)[0]
idx_MZI_mins = find_peaks(-MZI_crop, height=-MZI_min_threshold, distance=MZI_mean_idx_distance)[0]
remove_i_peaks = 10
remove_f_peaks = 10
idx_MZI_maxs_and_mins = np.sort(np.concatenate((idx_MZI_maxs, idx_MZI_mins), axis=0))[remove_i_peaks:(-remove_f_peaks)]

fig, ax = plt.subplots(figsize=(14,3))

ax.plot(MZI_crop)
ax.scatter(idx_MZI_maxs_and_mins, MZI_crop[idx_MZI_maxs_and_mins], c='r')
ax.set_xlim(40000,50500)

In [8]:
i=18
FSR_nm = []
for i in range(len(idx_HCN_crop_mins)-1):
    MZI_i_idx = np.argmin(np.abs(idx_MZI_maxs_and_mins-idx_HCN_crop_mins[i]))
    MZI_f_idx = np.argmin(np.abs(idx_MZI_maxs_and_mins-idx_HCN_crop_mins[i+1]))
    num_peaks = len(idx_MZI_maxs_and_mins[MZI_i_idx:MZI_f_idx+1])
    
    HCN_i_value = HCN_values[i]
    HCN_f_value = HCN_values[i+1]
    pitch = (HCN_f_value-HCN_i_value)/num_peaks
    
    FSR_nm.append(2*pitch)
FSR_nm = np.array(FSR_nm)
FSR_nm_interp = interp1d((np.array(HCN_values[:-1])+np.array(HCN_values[1:]))/2, FSR_nm)

In [ ]:
λ = (np.array(HCN_values[:-1])+np.array(HCN_values[1:]))/2
c = 299792458
FSR_MHz = 1000*(c/(λ*λ))*FSR_nm_interp(λ)
FSR_MHz